# Ground truth

Three different ground truths were prepared by three different people and then compared to increase the reliability of the ground truth.

In [1]:
%matplotlib tk

import hyperspy as hs
import pyxem as px
from matplotlib import pyplot as plt
#matplotlib.use("Agg")
import numpy as np
import os
from tqdm import tqdm
from skimage import restoration
from matplotlib.colors import LogNorm

from numpy import log10

from scipy.ndimage import binary_fill_holes, label
from scipy import ndimage as ndi
from scipy.ndimage import gaussian_filter

from skimage.feature import peak_local_max
from skimage import data
from skimage import img_as_float
from skimage.morphology import reconstruction
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from skimage.filters import sobel, threshold_triangle, threshold_li

## Load dataset

In [2]:
signal = hs.io.load('SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3_preprocessed.hspy')
DP_scale = signal.axes_manager[2].scale
DP_scale

## Create ground truths

### Ground truth #1

In this approach, a virtual dark-field (VDF) image is created per precipitate phase electron diffraction pattern.
vdf_phase_x refers to the VDFs for the two edge on configurations of T1 and $\theta'$, while vdf_theta_face_on refers to the VDF from $\theta'$ viewed face on.

#### Step 1: Create VDFs based on one reflection per precipitate phase

In [23]:
vdf_T1_1 = signal.get_integrated_intensity(hs.roi.CircleROI(cx=-0.133035, cy=0.437115, r=0.0285075, r_inner=0))

vdf_T1_2 = signal.get_integrated_intensity(hs.roi.CircleROI(cx=0.437115, cy=0.11403, r=0.0285075, r_inner=0))

vdf_theta1 = signal.get_integrated_intensity(hs.roi.CircleROI(cx=0.13804, cy=-0.2618, r=0.02856, r_inner=0))

vdf_theta2 = signal.get_integrated_intensity(hs.roi.CircleROI(cx=-0.019005, cy=0.285075, r=0.0285075, r_inner=0.00950249))

vdf_theta_faceon = signal.get_integrated_intensity(hs.roi.CircleROI(cx=0.323085, cy=0.0950249, r=0.0285075, r_inner=0))

In [9]:
vdf_theta_faceon.plot()

In [21]:
plt.close('all')
roi=hs.roi.CircleROI(cx=0.133035, cy=-0.26607, r=0.0285075, r_inner=0)
signal.plot_integrated_intensity(roi=roi)

In [22]:
roi

CircleROI(cx=0.13804, cy=-0.2618, r=0.02856, r_inner=0)

In [66]:
vdf_theta_faceon.plot()

#### Step 2: Threshold each VDF

$\theta'$ edge on. As this phase has intense reflections, li- and triangle thresholding was deemed sufficient for creating the binary VDFs for this phase.

In [24]:
vdf_theta1_bin = vdf_theta1.data > threshold_triangle(vdf_theta1.data)
fig,ax = plt.subplots(1,2,sharex=True,sharey=True)
ax[0].imshow(vdf_theta1_bin)
ax[1].imshow(vdf_theta1, norm=LogNorm())

In [15]:
from skimage.filters import threshold_li
vdf_theta2_bin = vdf_theta2.data > threshold_triangle(vdf_theta2.data)
fig,ax = plt.subplots(2,1,sharex=True,sharey=True)
ax[0].imshow(vdf_theta2_bin)
ax[1].imshow(vdf_theta2, norm=LogNorm())

In [14]:
vdf_theta_bin = vdf_theta1_bin + vdf_theta2_bin
fig,ax = plt.subplots(2,1,sharex=True,sharey=True)
ax[0].imshow(vdf_theta_bin)
ax[1].imshow(vdf_theta1 + vdf_theta2, norm=LogNorm())

$\theta'$ face on. As this phase has weak refelctions, watershed segmentation was used for the creation of the binary VDFs for this phase.

In [38]:
type(vdf_theta_faceon.data)

numpy.ndarray

In [ ]:
markers = np.zeros_like(vdf_theta_faceon)
markers[vdf_theta_faceon < 250] = 1
markers[vdf_theta_faceon > 400] = 2

elevation_map = sobel(vdf_theta_faceon.data)

segmentation = watershed(elevation_map, markers)
segmentation = ndi.binary_fill_holes(segmentation-1)
vdf_theta_faceon_bin = segmentation > 0

fig,ax = plt.subplots(1,2,sharex=True,sharey=True)
ax[0].imshow(vdf_theta_faceon_bin)
ax[1].imshow(vdf_theta_faceon, norm=LogNorm())

T1 edge on. As this phase has weak reflections, watershed regmentation was used for the creation of the binary VDFs for this phase.

In [72]:
markers = np.zeros_like(vdf_T1_1)
markers[vdf_T1_1 < 250] = 1
markers[vdf_T1_1 > 300] = 2

elevation_map = sobel(vdf_T1_1.data)
segmentation = watershed(elevation_map, markers)
vdf_T1_1_bin = segmentation > 1
fig,ax = plt.subplots(1,2,sharex=True,sharey=True)
ax[0].imshow(vdf_T1_1_bin)
ax[1].imshow(vdf_T1_1, norm=LogNorm())

In [76]:
markers = np.zeros_like(vdf_T1_2)
markers[vdf_T1_2 < 250] = 1
markers[vdf_T1_2 > 300] = 2

elevation_map = sobel(vdf_T1_2.data)
segmentation = watershed(elevation_map, markers)
segmentation = ndi.binary_fill_holes(segmentation-1)
vdf_T1_2_bin = segmentation > 0

fig,ax = plt.subplots(1,2,sharex=True,sharey=True)
ax[0].imshow(vdf_T1_2_bin)
ax[1].imshow(vdf_T1_2, norm=LogNorm())

In [81]:
signal.plot(navigator=hs.signals.Signal2D(vdf_T1_bin))

In [80]:
vdf_T1_bin = vdf_T1_2_bin + vdf_T1_1_bin

#### Step 3: Remove unwanted pixels

$\theta'$

In [13]:
from skimage.measure import label, regionprops

theta_labeled = label(vdf_theta_bin)
props = regionprops(theta_labeled)
for i in range(len(props)):
    if props[i].area <= 3:
        for x_coord in props[i].coords[:,0]:
            for y_coord in props[i].coords[:,1]:
                vdf_theta_bin[x_coord, y_coord] = 0
plt.imshow(vdf_theta_bin)

In [14]:
markers = np.zeros_like(vdf_theta_faceon)
markers[vdf_theta_faceon < 600] = 1
markers[vdf_theta_faceon > 650] = 2

elevation_map = sobel(vdf_theta_faceon)

segmentation = watershed(elevation_map, markers)
segmentation = ndi.binary_fill_holes(segmentation-1)
vdf_theta_faceon_bin = segmentation > 0

#fig,ax = plt.subplots(1,2,sharex=True,sharey=True)
#ax[0].imshow(vdf_theta_faceon_bin)
#ax[1].imshow(vdf_theta_faceon, norm=LogNorm())

In [15]:
vdf_theta_faceon_labeled = label(vdf_theta_faceon_bin)
props = regionprops(vdf_theta_faceon_labeled)
for i in range(len(props)):
    if props[i].area <= 2000:
        for x_coord in props[i].coords[:,0]:
            for y_coord in props[i].coords[:,1]:
                vdf_theta_faceon_bin[x_coord, y_coord] = 0


vdf_theta_faceon_bin[np.argwhere(np.logical_and(vdf_theta_faceon_bin, vdf_theta_bin))[:,0], np.argwhere(np.logical_and(vdf_theta_faceon_bin, vdf_theta_bin))[:,1]] = False

vdf_theta_faceon_labeled = label(vdf_theta_faceon_bin)
props = regionprops(vdf_theta_faceon_labeled)
for i in range(len(props)):
    if props[i].area <= 2000:
        for x_coord in props[i].coords[:,0]:
            for y_coord in props[i].coords[:,1]:
                vdf_theta_faceon_bin[x_coord, y_coord] = 0


plt.imshow(vdf_theta_faceon_bin)

#### Create the ground truth image

Some of the precipitate phases are overlapping. A simple approach was chosen to overcome this challenge. Here, $\theta'$ edge on are prioritized over T1 edge on which is prioritized over $\theta'$ face on.

In [16]:
###### 
# Theta edge on trumps T1. T1 trumps theta face on #

vdf_T1_bin[np.argwhere(np.logical_and(vdf_T1_bin, vdf_theta_bin))[:,0], np.argwhere(np.logical_and(vdf_T1_bin, vdf_theta_bin))[:,1]] = False
vdf_theta_faceon_bin[np.argwhere(np.logical_and(vdf_T1_bin, vdf_theta_faceon_bin))[:,0], np.argwhere(np.logical_and(vdf_T1_bin, vdf_theta_faceon_bin))[:,1]] = False


vdf_T1_bin3 = np.zeros_like(vdf_T1_bin, dtype='int')
vdf_T1_bin3[vdf_T1_bin > 0] = 3

vdf_theta_bin_faceon2 = np.zeros_like(vdf_theta_faceon_bin, dtype='int')
vdf_theta_bin_faceon2[vdf_theta_faceon_bin > 0] = 2

vdf_theta_bin1 = np.zeros_like(vdf_theta_bin, dtype='int')
vdf_theta_bin1[vdf_theta_bin > 0] = 1

ground_truth_1 = vdf_T1_bin3 + vdf_theta_bin_faceon2 + vdf_theta_bin1
ground_truth_1 = hs.signals.BaseSignal(ground_truth_1)

### Correct some pixels manually:
ground_truth_1.isig[147,130] = 0

plt.imshow(ground_truth_1)

### Ground truth #2

In [4]:
signal_bin = signal.rebin(scale=(1, 1, 2, 2))
shift = 1
signal_log = log10(signal_bin.data+shift) - log10(shift)
signal_log = px.signals.ElectronDiffraction2D(signal_log)
scale = 1
signal_log.set_diffraction_calibration(scale)
signal_log.data = signal_log.data/np.max(signal_log.data)*4096
signal_log.change_dtype('uint32')

signal_log.plot()

#### Create binary image of $\theta'$ edge on:

In [18]:
##################
# Mask the theta' viewed edge on
##################
roi = hs.roi.CircleROI(cx=7, cy=-15, r=2.5, r_inner=0)
righttop = signal_log.get_integrated_intensity(roi)
#righttop.plot(cmap='magma_r')
righttop_mask = righttop.data.copy()
righttop_mask[righttop_mask<32000] = 0
#hs.signals.BaseSignal(righttop_mask).plot(cmap='magma_r')
righttop_mask = righttop_mask.astype('bool')

In [19]:
roi = hs.roi.CircleROI(cx=-14, cy=-8, r=2.5, r_inner=0)
lefttop = signal_log.get_integrated_intensity(roi)
#lefttop.plot(cmap='magma_r')
lefttop_mask = lefttop.data.copy()
lefttop_mask[lefttop_mask<32100] = 0
#hs.signals.BaseSignal(lefttop_mask).plot(cmap='magma_r')
lefttop_mask = lefttop_mask.astype('bool')

In [20]:
vdf_theta_bin = np.array(righttop_mask.copy() + lefttop_mask.copy(), dtype='bool')
hs.signals.BaseSignal(vdf_theta_bin).plot(cmap='magma_r')

#### Create binary image of T1:

In [21]:
from scipy.ndimage import label, distance_transform_edt
#################
# Mask the T1 phases
#################
roi = hs.roi.CircleROI(cx=6, cy=-26, r=2.5, r_inner=0)
righttop = signal_log.get_integrated_intensity(roi)
#righttop.plot(cmap='magma_r')
righttop_mask = righttop.data.copy()
righttop_mask[righttop_mask<27300] = 0
#hs.signals.BaseSignal(righttop_mask).plot(cmap='magma_r')
righttop_mask[vdf_theta_bin] = 0
#hs.signals.BaseSignal(righttop_mask).plot(cmap='magma_r')
labs, _ = label(righttop_mask)
#hs.signals.BaseSignal(labs).plot(cmap='magma_r')
# remove the labelled regions with sizes below a threshold
th = 10
for i in np.arange(1, np.max(labs)):
    if np.sum(labs[labs == i])/i < th:
        righttop_mask[labs == i] = 0
#hs.signals.BaseSignal(righttop_mask).plot(cmap='magma_r')
righttop_mask = righttop_mask.astype('bool')

In [22]:
roi = hs.roi.CircleROI(cx=-24, cy=-9, r=2.5, r_inner=0)
lefttop = signal_log.get_integrated_intensity(roi)
#lefttop.plot(cmap='magma_r')
lefttop_mask = lefttop.data.copy()
lefttop_mask[lefttop_mask<27000] = 0
#hs.signals.BaseSignal(lefttop_mask).plot(cmap='magma_r')
lefttop_mask[vdf_theta_bin] = 0
#hs.signals.BaseSignal(lefttop_mask).plot(cmap='magma_r')
labs, _ = label(lefttop_mask)
#hs.signals.BaseSignal(labs).plot(cmap='magma_r')
# remove the labelled regions with sizes below a threshold
th = 10
for i in np.arange(1, np.max(labs)):
    if np.sum(labs[labs == i])/i < th:
        lefttop_mask[labs == i] = 0
#hs.signals.BaseSignal(lefttop_mask).plot(cmap='magma_r')
lefttop_mask = lefttop_mask.astype('bool')

vdf_T1_bin = np.array(righttop_mask.copy() + lefttop_mask.copy(), dtype='bool')
hs.signals.BaseSignal(vdf_T1_bin).plot(cmap='magma_r')

#### Create binary image of $\theta'$ viewed face on:

In [23]:
###################
# Mask of the theta' viewed face on.
###################
roi = hs.roi.CircleROI(cx=3, cy=-19, r=2.5, r_inner=0.5)
blobs = signal_log.get_integrated_intensity(roi)
#blobs.plot(cmap='magma_r')
blobs_mask = blobs.data.copy()
blobs_mask[blobs_mask<29000] = 0
#hs.signals.BaseSignal(blobs_mask).plot(cmap='magma_r')
blobs_mask_sobel = sobel(blobs_mask)
#hs.signals.BaseSignal(blobs_mask_sobel).plot(cmap='magma_r')
dists = distance_transform_edt(blobs_mask)
#hs.signals.BaseSignal(dists).plot(cmap='magma_r')
peaks = peak_local_max(dists, min_distance = 10, exclude_border = 1, threshold_abs = 18)
peaks_mask = np.zeros_like(blobs_mask)
peaks_mask[tuple(peaks.T)] = 1
#hs.signals.BaseSignal(peaks_mask).plot(cmap='magma_r')
seg = watershed(image=blobs_mask_sobel, markers = peaks_mask, mask=blobs_mask)
#hs.signals.BaseSignal(seg).plot(cmap='magma_r')
seg_fill = binary_fill_holes(seg)
#hs.signals.BaseSignal(seg_fill).plot(cmap='magma_r')
# Manually remove the incorrectly labeled regions around the 'duck'
seg_fill[58, 95] = 0
seg_fill[59, 95] = 0
seg_fill[60, 94] = 0
seg_fill[61, 93] = 0
seg_fill[62, 93] = 0
seg_fill[47, 86] = 0
seg_fill[46, 87] = 0
seg_fill[46, 88] = 0
labels, num = label(seg_fill)
#hs.signals.BaseSignal(labels).plot(cmap='magma_r')
labels[labels == 3] = 0
labels[labels == 1] = 0
hs.signals.BaseSignal(labels).plot(cmap='magma_r')

vdf_theta_faceon_bin = np.array(labels, dtype='bool')

In [24]:
# NB!! The needles are prioritized over plates that are prioritized over blobs"
ground_truth_2 = np.zeros_like(blobs, dtype='int32')
ground_truth_2[vdf_theta_faceon_bin] = 2
ground_truth_2[vdf_T1_bin] = 3
ground_truth_2[vdf_theta_bin] = 1
ground_truth_2 = hs.signals.BaseSignal(ground_truth_2)

In [25]:
# Correct some pixels manually:
ground_truth_2.data[15, 20] = 0 #(from 3 to 0)
ground_truth_2.data[71, 39] = 3 #(from 0 to 3)
ground_truth_2.data[45, 65] = 2 #(from 0 to 2)
ground_truth_2.data[45, 64] = 2 #(from 0 to 2)
ground_truth_2.data[47, 64] = 2 #(from 0 to 2)
ground_truth_2.data[47, 65] = 2 #(from 0 to 2)
ground_truth_2.data[52, 92] = 2 #(from 1 to 2)
ground_truth_2.data[188, 92] = 3 #(from 0 to 3)
ground_truth_2.data[186, 96] = 3 #(from 0 to 3)
ground_truth_2.data[185, 98] = 3 #(from 0 to 3)
ground_truth_2.data[301, 153] = 1 #(from 0 to 1)
ground_truth_2.data[303, 157] = 1 #(from 0 to 1)
ground_truth_2.data[20, 330] = 2 #(from 1 to 2)
ground_truth_2.data[21, 331] = 2 #(from 1 to 2)
ground_truth_2.data[477, 453] = 1 #(from 0 to 1)
ground_truth_2.data[478, 454] = 1 #(from 0 to 1)


ground_truth_2.plot(cmap='viridis')
#plt.savefig(folder+'ground_truth_Tina.png')
#ground_truth.save(folder+'ground_truth_Tina.hdf5')

signal_log.plot(navigator=ground_truth_2, cmap='magma_r')

### Ground truth #3

#### Make Precipitate object

In [3]:
from scipy.ndimage import label
import copy

#Creating empty dataset for variables in Precipitates() and also combined ground truth map eventually
empty_vdf = signal.get_integrated_intensity(hs.roi.CircleROI(cx=0, cy=0, r=0.5)).data.copy()
empty_vdf[empty_vdf > 0] = 0 

class Precipitate():
    name = ""
    th = 0
    roi = hs.roi.CircleROI(cx=0, cy=0, r=0, r_inner=0)
    roi2 = hs.roi.CircleROI(cx=0, cy=0, r=0, r_inner=0)
    label_th = 0
    vdf = empty_vdf.copy()
    vdf_mask = empty_vdf.copy()
    label, _ = label(vdf_mask)

    def __init__(self, name, th, label_th, roi):
        self.name = name
        self.th = th
        self.roi = roi
        self.label_th = label_th

def make_precip(name, th, label_th, roi):
    precipitate = Precipitate(name, th, label_th, roi)
    return precipitate
def make_precip2(name, th, label_th, roi, roi2):
    precipitate = Precipitate(name, th, label_th, roi)
    precipitate.roi2 = roi2
    return precipitate

In [4]:
#In plane theta'. Used two reflections for better signal 
roi_thetaPrimes = hs.roi.CircleROI(cx=-0.114238, cy=0.323675, r=0.0475943, r_inner=0)
roi_thetaPrimes2 = hs.roi.CircleROI(cx=0.323675, cy=0.0761587, r=0.0475943, r_inner=0)

#fat horizontals (T1)
roi_h = hs.roi.CircleROI(cx=-0.47124, cy=-0.14756, r=0.02856, r_inner=0)

#fat verticals (T1)
roi_v = hs.roi.CircleROI(cx=0.119, cy=-0.47124, r=0.02856, r_inner=0)

#left angled (thetaPrime, (200)_[010])
roi_la = hs.roi.CircleROI(cx=0.2142, cy=0.13804, r=0.0476, r_inner=0)

#right angled (thetaPrime, (020)_[100])
roi_ra = hs.roi.CircleROI(cx=0.13328, cy=-0.2856, r=0.0476, r_inner=0)

#--------------------------------------------------------------------------------------------#
#       inputs        |      name       |  th, | label_th,|      roi,     |     roi2         #
#--------------------------------------------------------------------------------------------#
Theta_3 =make_precip2("Theta, in-plane", 0.2,      782,    roi_thetaPrimes, roi_thetaPrimes2)#
T1_1    = Precipitate("T1, horizontal",  0.0138,    28,          roi_h )                     #
T1_2    = Precipitate("T1, vertical",    0.014,     28,          roi_v )                     #
Theta_1 = Precipitate("Theta, lefty",    0.12,       7,          roi_la)                     #
Theta_2 = Precipitate("Theta, righty",   0.17,       5,          roi_ra)                     #
#--------------------------------------------------------------------------------------------#

#list:
precips   = np.asarray([Theta_3, T1_1, T1_2, Theta_1, Theta_2])
numbering = np.asarray([   2,     3,     3,     1,       1   ])

In [6]:
for precip in precips:
    if (precip.name != "Theta, in-plane"): #in plane theta use two rois
        precip.vdf = signal.get_integrated_intensity(precip.roi)
    else:
        precip.vdf = signal.get_integrated_intensity(precip.roi) + signal.get_integrated_intensity(precip.roi2)

#### Create masks where you remove everything below certain threshold

In [7]:
for i, precip in enumerate(precips):
    if (precip.name != "Theta, in-plane"):
        vdf_mask = precip.vdf.data.copy()
        vdf_mask[vdf_mask < precip.th] = 0
        vdf_mask[vdf_mask > 0 ] = numbering[i]
        precip.vdf_mask = vdf_mask

    else:
        ### Need to remove edge on theta from in plane theta:
        vdf_mask_la = precips[3].vdf.data.copy()
        vdf_mask_la[vdf_mask_la < precips[3].th] = 0 # 3= left angle theta needles (see parameters for precips)
        vdf_mask_la[vdf_mask_la > 0] = 1

        vdf_mask_ra = precips[4].vdf.data.copy()
        vdf_mask_ra[vdf_mask_ra < precips[4].th] = 0 # 4 = righ angle theta needles
        vdf_mask_ra[vdf_mask_ra > 0] = 1

        #Adding unwanted thetas to one mask and setting any potential 2 = 1
        combined_mask = vdf_mask_la + vdf_mask_ra
        combined_mask[combined_mask > 0] = 1 # All needle pix should be 1
        #### ----- ###

        vdf_mask = precip.vdf.data.copy()
        vdf_mask[vdf_mask < precip.th] = 0
        vdf_mask[vdf_mask > 0 ] = numbering[i]

        # Removing other oriientations from map
        vdf_mask[vdf_mask == combined_mask*2] = 0
        precip.vdf_mask = vdf_mask

#### Use label from scipy to remove noise

In [8]:
from scipy.ndimage import label

for precip in precips:
    # Labels groups of pixels by size
    precip.label = label(precip.vdf_mask)[0]
    for i in np.arange(1, np.max(precip.label)):
        # Everything below label_th size concidered noise and removed:
        if np.sum(precip.label[precip.label == i])/i < precip.label_th: 
            precip.vdf_mask[precip.label == i] = 0

#### Now combine all masks to ground truth

In [9]:
ground_truth_3 = empty_vdf.copy()

for i, precip in enumerate(precips):
    precip = precip
    ground_truth_3[precip.vdf_mask == numbering[i]] = numbering[i]
    
plt.figure()
plt.imshow(ground_truth_3)

ground_truth_3 = hs.signals.BaseSignal(ground_truth_3)

### Save ground truths

In [33]:
ground_truth_1.axes_manager[0].scale = 4.63
ground_truth_1.axes_manager[1].scale = 4.63
ground_truth_1.axes_manager[0].units = 'nm'
ground_truth_1.axes_manager[1].units = 'nm'

ground_truth_2.axes_manager = ground_truth_1.axes_manager
ground_truth_3.axes_manager = ground_truth_1.axes_manager

In [34]:
directory = '/Ground_truths/'

if not os.path.exists(directory):
    os.makedirs(directory)

ground_truth_1.save(directory + 'ground_truth_1.hdf5')
ground_truth_2.save(directory + 'ground_truth_2.hdf5')
ground_truth_3.save(directory + 'ground_truth_3.hdf5')

Overwrite 'H:\Ground_truths\ground_truth_1.hdf5' (y/n)?
 y
Overwrite 'H:\Ground_truths\ground_truth_2.hdf5' (y/n)?
 y
Overwrite 'H:\Ground_truths\ground_truth_3.hdf5' (y/n)?
 y


### Load and compare the three different ground truths

The following ints are used for the four different precipitate phases in the ground truths: <br>
<ul>
    <li> Al: 0 </li>
    <li> $\theta'$ edge on: 1 </li>
    <li> $\theta'$ face on: 2 </li>
    <li> T1: 3 </li>
    <li> Not indexed: 4 </li>
</ul>

In [35]:
directory = '/Ground_truths/'

ground_truth_1 = hs.io.load(directory + 'ground_truth_1.hdf5')
ground_truth_2 = hs.io.load(directory + 'ground_truth_2.hdf5')
ground_truth_3 = hs.io.load(directory + 'ground_truth_3.hdf5')

In [291]:
difference_12 = ground_truth_1.data - ground_truth_2.data
difference_13 = ground_truth_1.data - ground_truth_3.data
difference_23 = ground_truth_2.data - ground_truth_3.data

In [292]:
fig, ax = plt.subplots(1,3, sharex=True, sharey=True)
ax[0].imshow(difference_12)
ax[0].set_title('Difference 1 2')
ax[1].imshow(difference_13)
ax[1].set_title('Difference 1 3')
ax[2].imshow(difference_23)
ax[2].set_title('Difference 2 3')

Text(0.5, 1.0, 'Difference 2 3')

### Prepare the final ground truth image

In [83]:
gts = hs.signals.Signal2D([ground_truth_1.data, ground_truth_2.data, ground_truth_3.data])

hs.utils.plot.plot_images(gts)

[<AxesSubplot:title={'center':' (0,)'}>,
 <AxesSubplot:title={'center':' (1,)'}>,
 <AxesSubplot:title={'center':' (2,)'}>]

In [37]:
ground_truth = np.zeros_like(ground_truth_1.data)
for i in range(ground_truth_1.data.shape[0]):
    for j in range(ground_truth_1.data.shape[1]):
        if ground_truth_1.data[i,j] == ground_truth_2.data[i,j]: 
            ground_truth[i,j] = ground_truth_1.data[i,j]
        if ground_truth_1.data[i,j] == ground_truth_3.data[i,j]:
            ground_truth[i,j] = ground_truth_1.data[i,j]
        if ground_truth_2.data[i,j] == ground_truth_3.data[i,j]:
            ground_truth[i,j] = ground_truth_3.data[i,j]
        if ground_truth_1.data[i,j] != ground_truth_2.data[i,j] and ground_truth_1.data[i,j] != ground_truth_3.data[i,j] and ground_truth_2.data[i,j] != ground_truth_3.data[i,j]:
            ground_truth[i,j] = 4
plt.imshow(ground_truth)

In [38]:
from matplotlib.colors import to_rgba
from matplotlib.colors import LinearSegmentedColormap

color_names = ['linen', 'darkorange', 'dodgerblue', 'forestgreen', 'red']
colors = [to_rgba(c) for c in color_names]

cmap = LinearSegmentedColormap.from_list('gt_cmap', colors, N=len(color_names))

plt.figure()
plt.imshow(ground_truth.data, cmap=cmap)

In [71]:
ground_truth = hs.signals.BaseSignal(ground_truth)
ground_truth.axes_manager = ground_truth_1.axes_manager.deepcopy()

In [87]:
from matplotlib_scalebar.scalebar import ScaleBar
scalebar = ScaleBar(4.63, 'nm', location = 'lower left')

from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

fg_color = 'white'
bg_color = 'black'

cmappable = ScalarMappable(norm=Normalize(0,1), cmap=cmap)
#cmappable.set_array([lower-deltac, upper+deltac])

fig,ax = plt.subplots()
ax.axis('off')
ax.imshow(ground_truth, cmap=cmap)
ax.add_artist(scalebar)
#ax.legend(['Al', 'Theta edge on', 'Theta face on', 'T1'])
cbar = plt.colorbar(cmappable)

tick_locs = ((np.arange(N) + 0.5))/N
cbar.set_ticks(tick_locs)

cbar.ax.set_yticklabels(['Al', r'$\theta$ edge on', r'$\theta$ face on', 'T1', 'Not indexed'])

plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color=fg_color, fontsize=8)

plt.show()
plt.savefig(directory + 'Ground_truth_all.png', transparent = True, bbox_inches = 'tight', pad_inches = 0, dpi=600)
ground_truth.save(directory + 'Ground_truth_all.hdf5')